In [7]:
'''
código que se uso para generar los usarios fake 
'''


from faker import Faker
import numpy as np 
import pandas as pd

# importación de librerías propias
import utils.folders_tb as fold 
import utils.tools as tools
from utils.mysql_driver import MySQL
import utils.mysql_driver as drv

fake = Faker('es_ES')
df_localidades = fold.read_file_toDF('CP_SP_LatLon',',')
df_localidades.drop(columns='Unnamed: 0', inplace=True)


In [10]:

df_localidades

,CA,provincia,poblacion,CP,lat,lon
0,Cataluña,Barcelona,Barcelona,8001,"41,38424664","2,17634927"
1,Cataluña,Barcelona,Barcelona,8002,"41,38424664","2,17634927"
2,Cataluña,Barcelona,Barcelona,8003,"41,38424664","2,17634927"
3,Cataluña,Barcelona,Barcelona,8004,"41,38424664","2,17634927"
4,Cataluña,Barcelona,Barcelona,8005,"41,38424664","2,17634927"
...,...,...,...,...,...,...
319,Castilla la Mancha,Toledo,Toledo,45004,"39,85715187","-4,02431421"
320,Castilla la Mancha,Toledo,Toledo,45005,"39,85715187","-4,02431421"
321,Castilla la Mancha,Toledo,Toledo,45006,"39,85715187","-4,02431421"
322,Castilla la Mancha,Toledo,Toledo,45007,"39,85715187","-4,02431421"


In [6]:
df_localidades['CP'] = df_localidades['codigopostalid'].apply(lambda x: str(x) if x > 10000 else '0'+str(x)) #se convierte el CP original en string 
df_localidades.drop(columns='codigopostalid', inplace=True)
df_localidades['CA'] = df_localidades['CP'].apply(lambda x: tools.CP_toCom(x))
df_localidades = df_localidades[['CA','provincia','poblacion','CP','lat','lon']]
fold.save_df_to_csv(df_localidades,'final_loc')


In [13]:
df_localidades['CP'][25]

8026

In [16]:
customers_list = []
discap_list = ['motora','visual','auditiva','cognitiva','orgánica','ninguna']
columnas_BDD = ['perfil','nombre','apellidos','email','password','token','dni','direc','CP','ciudad','provincia','CA', 'lat','lon','dicapacidad','grado']

for _ in range(20):

    user_fields = []
    # Generación del perfil, nombre y appelidos
    user_fields.append('usuario')
    user_fields.append(fake.first_name())
    apellidos = fake.last_name() + ' ' + fake.last_name()
    user_fields.append(apellidos)

    # Generación datos de alta
    user_fields.append(fake.email())
    user_fields.append(fake.password(digits=True, upper_case=True, lower_case=True, length=10))
    user_fields.append(fake.windows_platform_token())
    user_fields.append(fake.nif())
    
    # Generación de los campos de la dirección
    address_seed = fake.address()
    address_seed_list = list(address_seed.split())
    address_CP = address_seed_list[-1]
    address_city = address_seed_list[-2].replace(',', '')
    address_clean = address_seed.replace(address_CP, '')
    address_clean = address_clean.replace(address_city, '')
    address_clean = address_clean.replace('\n', '')
    address_clean = address_clean.replace(',', '')
    user_fields.append(address_clean)

    # Generador aleatorio de ubicación
    rd_seed = np.random.randint (0,325)
    user_fields.append(df_localidades['CP'][rd_seed])
    user_fields.append(df_localidades['poblacion'][rd_seed])
    user_fields.append(df_localidades['provincia'][rd_seed])
    user_fields.append(df_localidades['CA'][rd_seed])
    user_fields.append(df_localidades['lat'][rd_seed])
    user_fields.append(df_localidades['lon'][rd_seed])

    # Generación de la discapacidad y porcentaje
    discap = discap_list[np.random.randint(0,6)]
    user_fields.append(discap)
    if discap != 'ninguna':
        porcen_disc = np.random.randint(1,11)*3 + np.random.randint(1,21)+ np.random.randint(1,51)
        user_fields.append(porcen_disc)
    else:
        rand_usu = np.random.randint(1,101)
        if rand_usu >= 95:
            user_fields[0] = 'certificador'
        else:
            user_fields.append(0)

    # Integración del usuario generado en la lista de usuarios
    customers_list.append(user_fields)


df_users = pd.DataFrame(customers_list, columns= columnas_BDD)
#df_users = pd.DataFrame(customers_list, columns= ['nombre','apellido 1','apellido 2','direc','CP','ciudad'])
df_users

,perfil,nombre,apellidos,email,password,token,dni,direc,CP,ciudad,provincia,CA,lat,lon,dicapacidad,grado
0,usuario,Maxi,Roig Aguilera,salud72@gmail.com,If^5oIaMH!,Windows NT 10.0,03725491C,Camino Federico Montenegro 10Las,28025,Madrid,Madrid,Comunidad de Madrid,"40,40841191","-3,68760088",visual,35
1,usuario,Ana Belén,Mayol Molina,pinydia@yahoo.com,@m5M2WTdSe,Windows NT 6.2,27037402M,Callejón Azucena Pino 7 Apt. 31,23004,Jaén,Jaén,Andalucía,"37,7651913","-3,7903594",cognitiva,30
2,usuario,Samuel,Sierra Pedrero,faustomanzanares@botella.es,a$0Eh6zfkD,Windows NT 6.1,26041968B,Paseo Mateo Recio 61,36155,Pontevedra,Pontevedra,Galicia,"42,43158576","-8,643672092",visual,44
3,usuario,Rafael,Gelabert Larrañaga,carlos33@hotmail.com,^emr^Kbm)9,Windows NT 6.0,09775269Q,Rambla de Remedios Calzada 730 Apt. 35,36154,Pontevedra,Pontevedra,Galicia,"42,43158576","-8,643672092",orgánica,68
4,usuario,Borja,Ramírez Teruel,miroeliana@juan-muro.es,#zDfD2clW1,Windows 98,16998075V,Via de Dimas Piña 186 Puerta 2,8015,Barcelona,Barcelona,Cataluña,"41,38424664","2,17634927",motora,40
5,usuario,Clotilde,Aguilera Ferrer,rbernat@yahoo.com,Ad^O_K@X(3,Windows 98,22058060W,Pasadizo de Ale Planas 84,8012,Barcelona,Barcelona,Cataluña,"41,38424664","2,17634927",orgánica,45
6,usuario,Clara,Cáceres Valbuena,chemaporras@yahoo.com,k66YQ^qc+e,Windows NT 5.2,50460752V,Plaza Elías Pina 23 Piso 1,36002,Pontevedra,Pontevedra,Galicia,"42,43158576","-8,643672092",orgánica,52
7,usuario,María Teresa,Sandoval Ávila,eduardoayuso@mena.com,n^Q2Gjv(d(,Windows NT 6.1,51120665Z,Acceso de Xiomara Urrutia 21 Apt. 20,33174,Oviedo,Asturias,Principado de Asturias,"43,36232165","-5,84372206",motora,64
8,usuario,Antonia,Solano Camino,arribasvaleria@valderrama.net,@5M5XRzJ+4,Windows NT 6.2,92602510W,Via de María Pilar Aznar 66 Puerta 0,41008,Sevilla,Sevilla,Andalucía,"37,38620512","-5,99251368",auditiva,54
9,usuario,Soraya,Mayol Pulido,graciano18@coloma.org,(1IC)Uh5TS,Windows 95,21714270Q,Vial de Judith Serrano 8,8003,Barcelona,Barcelona,Cataluña,"41,38424664","2,17634927",motora,47


In [17]:
fold.save_df_to_csv(df_localidades,'usuarios')


In [7]:
for i in range (100):
    porcen_disc = np.random.randint(1,11)*3 + np.random.randint(1,71)
    print(porcen_disc)

58
18
66
66
81
62
68
40
12
52
27
52
57
37
42
85
45
52
59
41
43
16
37
79
79
43
49
23
35
51
57
30
67
90
59
30
96
66
55
59
27
66
65
72
65
26
41
60
38
33
64
48
41
85
79
20
18
37
79
48
74
24
46
17
22
85
79
44
92
28
74
89
77
17
86
32
88
83
33
51
41
64
62
79
12
82
97
55
61
69
23
35
46
93
32
29
39
17
72
19
